In [2]:
!pip install --upgrade langchain langchain-core langchain-community langchain_ibm pydantic


  Using cached langchain-0.3.21-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_community-0.3.20-py3-none-any.whl.metadata (2.4 kB)
  Using cached langchain_text_splitters-0.3.7-py3-none-any.whl.metadata (1.9 kB)
Using cached langchain-0.3.21-py3-none-any.whl (1.0 MB)
Using cached langchain_community-0.3.20-py3-none-any.whl (2.5 MB)
Using cached langchain_text_splitters-0.3.7-py3-none-any.whl (32 kB)
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.0.2
    Uninstalling langchain-text-splitters-0.0.2:
      Successfully uninstalled langchain-text-splitters-0.0.2
  Attempting uninstall: langchain
    Found existing installation: langchain 0.1.13
    Uninstalling langchain-0.1.13:
      Successfully uninstalled langchain-0.1.13
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.0.38
    Uninstalling langchain-community-0.0.38:
      Successfully uninstalled langchain-com

In [35]:
import os
from dotenv import load_dotenv
from langchain_ibm import WatsonxLLM
from langchain.prompts import PromptTemplate

In [2]:
load_dotenv()
watsonx_apikey = os.getenv('WATSONX_APIKEY')
project_id =  os.getenv('PROJECT_ID')
url= os.getenv('URL')


In [ ]:
params_dic= {
    "decoding_method": "greedy",
    "min_new_tokens": 0,
    "max_new_tokens": 300,
    "repetition_penalty": 1,
    "stop_sequences": ["Vasco."]
}
llm = WatsonxLLM(
    model_id=
           'ibm/granite-3-8b-instruct',
            
    url=url,
    apikey=watsonx_apikey,
    project_id=project_id,
    params=params_dic
)

In [ ]:
# modelos_granite_disponibles = [
#     'ibm/granite-13b-instruct-v2', 
#     'ibm/granite-20b-code-instruct', 
#     'ibm/granite-20b-multilingual', 
#     'ibm/granite-3-2-8b-instruct', 
#     'ibm/granite-3-2b-instruct', 
#     'ibm/granite-3-8b-instruct', 
#     'ibm/granite-34b-code-instruct', 
#     'ibm/granite-3b-code-instruct', 
#     'ibm/granite-8b-code-instruct', 
#     'ibm/granite-guardian-3-2b', 
#     'ibm/granite-guardian-3-8b', 
#     'ibm/granite-vision-3-2-2b'
# ]

In [6]:
llm.invoke("hello, whats your name?")

"\n\nHello! I'm an assistant developed to help answer your questions. I don't have a personal name, but you can call me Assistant. How can I assist you today?"

In [22]:
# Json obtenido del pipeline de analyze_water_data
total_water_data= {
    "city": "DONOSTIA-SAN SEBASTIAN",
    "water_tank_name": "Deposito de Putzueta",
    "alert": "KO",
    "component": [
        {
            "name": "CLORO LIBRE RESIDUAL",
            "current_value": 0.706,
            "lower_threshold": 0.598,
            "upper_threshold": 0.727,
            "median": 0.669,
            "component_alert": "OK"
        },
        {
            "name": "CONDUCTIVIDAD (20ºC)",
            "current_value": 118.411,
            "lower_threshold": 120.488,
            "upper_threshold": 129.752,
            "median": 124.981,
            "component_alert": "KO"
        },
        {
            "name": "PH (20ºC)",
            "current_value": 7.700,
            "lower_threshold": 7.426,
            "upper_threshold": 7.642,
            "median": 7.548,
            "component_alert": "KO"
        },
        {
            "name": "TEMPERATURA",
            "current_value": 16.201,
            "lower_threshold": 11.151,
            "upper_threshold": 14.408,
            "median": 12.619,
            "component_alert": "KO"
        },
        {
            "name": "TURBIDEZ",
            "current_value": 0.019,
            "lower_threshold": 0.040,
            "upper_threshold": 0.150,
            "median": 0.084,
            "component_alert": "KO"
        }
    ]
}

In [23]:
water_data = {
    "city": total_water_data["city"],
    "water_tank_name": total_water_data["water_tank_name"],
    "component": [
        {
            "name": c["name"],
            "current_value": c["current_value"],
            "lower_threshold": c["lower_threshold"],
            "upper_threshold": c["upper_threshold"]
        }
        for c in total_water_data["component"]
        if c["component_alert"] == "KO"
    ]
}

In [ ]:
# prompt_text = """
# Given the following JSON file containing information about sensor readings from a water tank, including the city, the name of the water tank, the components being monitored, their acceptable thresholds, and their current measured values:

# {water_data}

# Write a formal email addressed to the City Council of {city_name}. The email should clearly and professionally state that we have analyzed the sensor data from the specified water tank and identified that some parameters have exceeded their acceptable thresholds. Mention the specific components affected, their current values, and the corresponding thresholds. Indicate that this anomaly could be due to a potential sensor malfunction, and suggest conducting a technical inspection to verify the status of the equipment. The message should be concise, professional, and direct.

# End the email should be signed with the following line:
# "This email was generated by the AI agent of the Minsaiters team after analyzing water quality sensor data from Pais Vasco."
# """


In [32]:
prompt_text = """Analyze the JSON input containing water quality sensor data from a water tank in Pais Vasco. Write a formal, concise, and professional email body addressed to the city council of the specified city. The email should explain which components are outside their acceptable thresholds, specifying the component name, the current value, the threshold exceeded (upper or lower), and the direction of the anomaly (above or below threshold). Use a numbered list to display only the affected components. Do not include greetings, subject lines, or closings. End the message with the following line:

"This email was generated by the AI agent of the Minsaiters team after analyzing water quality sensor data from Pais Vasco."


Input: {{
  "city": "VITORIA",
  "water_tank_name": "Deposito de Bidebieta",
  "component": [
    {{
      "name": "CONDUCTIVIDAD (20ºC)",
      "current_value": 116.30,
      "lower_threshold": 120.48,
      "upper_threshold": 129.75
    }},
    {{
      "name": "PH (20ºC)",
      "current_value": 7.90,
      "lower_threshold": 7.42,
      "upper_threshold": 7.64
    }},
    {{
      "name": "TEMPERATURA",
      "current_value": 16.10,
      "lower_threshold": 11.15,
      "upper_threshold": 14.40
    }},
    {{
      "name": "TURBIDEZ",
      "current_value": 0.319,
      "lower_threshold": 0.040,
      "upper_threshold": 0.150
    }}
  ]
}}

Output:
Dear Vitoria City Council,

An analysis of the sensor data from the water tank Bidebieta in Vitoria has identified that some parameters have exceeded their acceptable thresholds. The affected components are:

1. CONDUCTIVIDAD (20ºC): The current value is 116.30, which is below the lower threshold of 120.48.
2. PH (20ºC): The current value is 7.90, which is above the upper threshold of 7.64.
3. TEMPERATURA: The current value is 16.10, which is above the upper threshold of 14.40.
4. TURBIDEZ: The current value is 0.319, which is above the upper threshold of 0.150.

These anomalies may be due to a potential sensor malfunction. We recommend performing a technical inspection to verify the condition of the equipment and ensure accurate data monitoring.

Best regards,

This email was generated by the AI agent of the Minsaiters team after analyzing water quality sensor data from País Vasco.


Input: {water_data}
Output:
"""


In [ ]:
def filter_water_data(total_water_data):
    water_data = {
    "city": total_water_data["city"],
    "water_tank_name": total_water_data["water_tank_name"],
    "component": [
        {
            "name": c["name"],
            "current_value": c["current_value"],
            "lower_threshold": c["lower_threshold"],
            "upper_threshold": c["upper_threshold"]
        }
        for c in total_water_data["component"]
        if c["component_alert"] == "KO"
    ]
    }
    return water_data

In [36]:
def ask_llm(llm, water_data):
    
    prompt_template = PromptTemplate(
            input_variables=["water_data"],
            template=prompt_text,
    )
 
    llm_chain = prompt_template | llm
    output = llm_chain.invoke({"water_data": water_data})
    return output

In [37]:
response=ask_llm(llm, water_data)

In [38]:
print(response)

Dear Donostia-San Sebastian City Council,

An analysis of the sensor data from the water tank Putzueta in Donostia-San Sebastian has identified that some parameters have exceeded their acceptable thresholds. The affected components are:

1. CONDUCTIVIDAD (20ºC): The current value is 118.411, which is below the lower threshold of 120.488.
2. PH (20ºC): The current value is 7.7, which is below the lower threshold of 7.426.
3. TEMPERATURA: The current value is 16.201, which is above the upper threshold of 14.408.
4. TURBIDEZ: The current value is 0.019, which is below the lower threshold of 0.04.

These anomalies may be due to a potential sensor malfunction. We recommend performing a technical inspection to verify the condition of the equipment and ensure accurate data monitoring.

Best regards,

This email was generated by the AI agent of the Minsaiters team after analyzing water quality sensor data from País Vasco.


In [ ]:
# def ask_llm(llm, water_data):
#     city_name = water_data["city"]
#     prompt_template = PromptTemplate(
#             input_variables=["water_data", "city_name"],
#             template=prompt_text,
#     )
 
#     # final_prompt = prompt_template.format(
#     #                                 water_data=water_data
#     #                                     , city_name=city_name
#     # )
    
#     # print("\n" + "="*50)
#     # print("🔍 PROMPT GENERADO:")
#     # print(final_prompt)
#     # print("="*50 + "\n")

#     llm_chain = prompt_template | llm
#     output = llm_chain.invoke({"water_data": water_data, "city_name": city_name})
#     return output
